In [ ]:
%pip install --upgrade pip 
%pip install -U langchain  
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph  
%pip install -qU "langchain[mistralai]" 
%pip install -qU langchain-huggingface
%pip install sentence-transformers
%pip install -qU langchain-core
%pip install -qU langchain_community pypdf pillow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


INDEXING

In [11]:
import getpass
import os
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from typing_extensions import List, TypedDict
from langgraph.graph import START, StateGraph
from dotenv import load_dotenv,find_dotenv

In [18]:
# langsmith e os e getpass para lidar com as chaves de api
load_dotenv(find_dotenv())

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key fot langsmith: ")

In [25]:
# chat model
if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

llm = init_chat_model("mistral-small-2503", model_provider = "mistralai")

In [ ]:
# embedding model
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

In [ ]:
# vector in-memory
vector_store = InMemoryVectorStore(embeddings)

In [ ]:
# Loading documents em pdf
directory_path = (
    "C:/JupyterNotebook/RAG/RAG_exames"
)
loader = PyPDFDirectoryLoader("RAG_exames/")

docs = loader.load()

In [ ]:
# Text splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split pdf into {len(all_splits)} sub-documents.")

In [ ]:
# Storing documents
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

INICIO DO RETRIEVAL E GENERATION

In [ ]:
# prompt
# fazer teste em txt
template = """Você é um especialista em análise de relatórios de mamografia. Por favor, leia o relatório quando indicado e extraia as seguintes informações:
Cisto:
- Presente ou Ausente
- Localização e tamanho do cisto
Nódulo:
- Presente ou Ausente
- Localização e tamanho do nódulo
Calcificação:
- Presente ou Ausente
- Localização e tamanho da calcificação
Microcalcificação:
- Presente ou Ausente
- Localização e tamanho da microcalcificação
BI-RADS: [valor]
Outras citações a avaliar: [observações adicionais relevantes]

Caso não encontre alguma informação que se encaixe, coloque [sem referência no texto].

Diretrizes de Interpretação

Diferenciação entre Nódulo e Cisto:

Se um achado é identificado inicialmente como "nódulo" na mamografia, mas confirmado como "cisto" no ultrassom, classifique apenas como CISTO (presente).
Nódulos são estruturas sólidas; cistos são estruturas predominantemente líquidas.
Complexos sólido-císticos devem ser reportados em ambas categorias (nódulo E cisto).


Priorização de Achados Múltiplos:

Quando houver múltiplos cistos/nódulos, reporte TODOS, priorizando:
a) Achados classificados como suspeitos pelo relatório
b) Achados de maior tamanho
c) Achados com características atípicas mencionadas


Diferenciação entre Calcificações e Microcalcificações:

Calcificações: estruturas maiores, geralmente descritas como "grosseiras", "distróficas", "vasculares"
Microcalcificações: estruturas menores, frequentemente descritas como "puntiformes", "pleomórficas", "lineares", "agrupadas", "em cluster"
Se o relatório mencionar "microcalcificações", classifique especificamente como microcalcificações
Se mencionar apenas "calcificações", classifique como calcificações.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
# estado para o langgraph
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [ ]:
# Funções do rag
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = custom_rag_prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [ ]:
# langgraph
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()